In [11]:
import torch 
import torch.nn as nn

import numpy as np
import pandas as pd

from torch.utils.data import Dataset, DataLoader

In [10]:
df = pd.read_csv('./data/gas.csv')
df.dropna()

,Date,Price
0,2025-03-03,3.80
1,2025-02-28,3.91
2,2025-02-27,3.91
3,2025-02-26,3.90
4,2025-02-25,3.88
...,...,...
7069,1997-01-13,4.00
7070,1997-01-10,3.92
7071,1997-01-09,3.61
7072,1997-01-08,3.80


In [ ]:
y = df['Price'][::-1].values

array([3.82, 3.8 , 3.61, ..., 3.91, 3.91, 3.8 ])

In [17]:
device = "cuda" if torch.cuda.is_available() else "cpu"

class DataSeries(Dataset):
    def __init__(self, seq, seq_length=10):
        self.seq_length = seq_length
        self.tot_samples = len(seq)

        self.seq = seq

    def __len__(self):
        return self.tot_samples
    
    def __getitem__(self, idx):
        X = self.seq[idx: idx+self.seq_length]
        Y = self.seq[idx+self.seq_length]
        X = torch.tensor(X, dtype=torch.float32).unsqueeze(-1).to(device)
        Y = torch.tensor(Y, dtype=torch.float32).to(device)

        return X, Y


torch.manual_seed(42)
fib_dataset1 = DataSeries(y, seq_length=10)

batch_size = 1
data_loader = DataLoader(fib_dataset1, batch_size, shuffle=True)


In [ ]:
class RNNModel(nn.Module):
    def __init__(self):
        super(RNNModel, self).__init__()
        
        self.rnn = nn.RNN(input_size=1,
                          hidden_size=8,
                          num_layers=5,
                          batch_first=True)
        
        self.fc1 = nn.Linear(in_features=8,
                             out_features=1,
                             bias=True)

        self.relu = nn.ReLU()
        
    def forward(self, x):
        
        output, status = self.rnn(x)
        output = output[:, -1, :]  # we need only the last one as our output
        
        output = self.relu(output)
        output = self.fc1(output)

        return output

model = RNNModel().to(device)
print(model)


RNNModel(
  (rnn): RNN(1, 8, num_layers=5, batch_first=True)
  (fc1): Linear(in_features=8, out_features=1, bias=True)
  (relu): ReLU()
)


In [19]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)
epochs = 5000

In [20]:
loss_list = []

for ep in range(epochs):

    rloss = 0
    model.train()

    for input, target in data_loader:
        input, target = input.to(device), target.to(device)

        ypred = model(input)

        loss = criterion(ypred, target)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        rloss += loss.item()
    
    rloss /= len(data_loader)
    loss_list.append(rloss)
    if ep%10 == 0:
        print(f"Ep {ep}/{epochs}: Loss = {rloss}")

ValueError: At least one stride in the given numpy array is negative, and tensors with negative strides are not currently supported. (You can probably work around this by making a copy of your array  with array.copy().) 